<a href="https://colab.research.google.com/github/mpfoster/Biochem6765/blob/master/Intro_plotting_fitting_python_6765.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to using Python for Biochem 6765 - SP20; Foster
This notebook is intended to serve as a tutorial guide for introducing students to data plotting and fitting in Python. It is by no means a complete description of capabilities or possibilities for data analysis in Python, but hopefully provides some useful guides and examples. 

Students should use the code cells as a guide, and feel free to copy/paste into their notebooks. The effect of editing the code can be seen by re-running the selected cell. Note that some cells will depend on execution of earlier cells ,though. In any case, the output fields should be cleared when starting this tutorial so that you can progress through it sequentially. This can be done at any time in Colab, via Tools > Command Palette > Clear all outputs.

Have fun! 2020-01-14 MPF

## Simple plotting and fitting with Python, using matplotlib and pandas
We will start with a cooperative isotherm of oxygen binding to hemoglobin. The data as follows, in which the first column is oxygen partial pressure (in mmHg), and the second is fractional saturation:
```
0.499	0.000
1.341	0.015
2.106	0.030
5.345	0.146
7.934	0.316
11.132	0.505
14.761	0.644
18.499	0.760
25.233	0.919
64.028	0.995
```
We plot the data using `matplotlib` https://matplotlib.org/ (a library with functionality similar to that of MATLAB), we define _x_ and _y_ data. This can be done manually, as ...

In [0]:
# import matplotlib library for MATLAB-like plotting
import matplotlib.pyplot as plt  # this library has the plotting tools
# enter the data manually
x = [0.499,  1.341,  2.106,  5.345,  7.934,  11.132,  14.761,  18.499,  25.233,
     64.028]
y = [0.0, 0.015, 0.03, 0.146, 0.316, 0.505, 0.644, 0.76, 0.919, 0.995]
# then plot the data: 
plt.plot(x,y, 'bo--') # 'bo--' means *b*lue 'o's and dashes
plt.xlabel('Oxygen Pressure (mm)')
plt.ylabel('Saturation')
#import numpy as np

Usually, our data is in some file. We can most easily read in the data using the `pandas` package, https://pandas.pydata.org/. Pandas simplifies loading tabular data into a "data frame"; basically a table with rows and columns. For this example, the data are available on the web as a space-separated text file, at this URL: https://github.com/mpfoster/Biochem6765/blob/master/data/hb+o2.txt. The first line of the file is a comment, so we will need to tell pandas to ignore lines that start with `comment='#'`, that the separator between fields is blank space `sep='\s+'`, and that the file does not have a reader row, `header=None`.

In addition to space, or tab-delimited files, pandas has utilites for reading `.csv` files (comma-separated-values; probably the most common), and can diretly read data from _Excel_ spreadsheets, or even on-line Google spreadsheets. See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_table.html

In [0]:
# use pandas to load the data from a url
import pandas as pd
df = pd.read_table("https://www.asc.ohio-state.edu/foster.281/gnuplot/gnuplot_tutorial1_files/hb+o2.txt",
                   sep='\s+', header=None, comment='#')
df.columns = ['PO2','Ys']  # rename columns; 0 is oxygen pressure; 1 is saturation
df  # print out the data frame

and... easily plot the data using funtions associated with a pandas data frame:

In [0]:
df.plot('PO2', 'Ys', kind='scatter', figsize=(3,3)) # plot the data using a scatter plot

NB: we can convert a column in a data frame to a list:

In [0]:
df['Ys'].tolist() # print a column as a list

## Fitting to a model
OK, we have loaded and plotted the binding data. It's common to interpret cooperative binding of multiple ligands to a macromolecule using the Hill equation: 

<img src="https://latex.codecogs.com/svg.latex?\Large&space;Y=\frac{[L]^n}{[L]^n+K_D^2}" title="\Large Y=\frac{[L]^n}{[L]^n+K_D^n}" />

where _n_ is the Hill coefficient. We will use the `lmfit` package https://lmfit.github.io/lmfit-py/ to generate the model and perfom fitting. `lmfit` is not a standard package in most python installations (it is built on other packages, especially `numpy` and `scipy`), and therefore needs to be installed. In a Jupyter Notebook, this can be accomplished by:
```
!pip install lmfit
```


In [0]:
# install lmfit
!pip install lmfit
from lmfit import Model, Parameter, report_fit  # import the desired components
import numpy as np  

In [0]:
def Hill(x, n, Kd): # the Hill equation
    return x**n/(x**n + Kd**n) 

x = df['PO2']
data = df['Ys']
#simdata = Hill(x,4,10)
#simdata
#plt.plot(x,simdata)

In [0]:
hmodel = Model(Hill, independent_vars=['x'])
hmodel
hmodel.param_names
result = hmodel.fit(data, x=x, n=4, Kd=1)
result

Next, we plot the data, along with the best-fit model, and the residuals; ALWAYS visually inspect the data, fit and residuals before interpreting parameters.

In [0]:
result.plot()

## Plotting multi-column data with pandas
Now, lets load some mulit-column data from the web, at this url: 
https://www.asc.ohio-state.edu/foster.281/gnuplot/gnuplot_tutorial1_files/example1.txt

In [0]:
import pandas as pd
df = pd.read_table("https://www.asc.ohio-state.edu/foster.281/gnuplot/gnuplot_tutorial1_files/example1.txt",sep='\s+', header=None, comment='#')
df.head()

We could issue a series of matplotlib sub-plot commands to separately plot the data in each column.... or, as here we will make use of built-in sub-plotting in pandas:

In [0]:
df.plot(x=0, y=[1,2,3], subplots='true', marker='o', figsize=[6,6]) # we specify which column has the x-axis, then a list of y axes to plot.

## Generating synthetic data using NumPy
Sometimes it's useful to simulate numerical data; numpy is just the right tool for this. We will start by plotting a simple sine function of the form _y = sin(x)_

In [0]:
# import necessary packages and libraries
import numpy as np
import matplotlib.pyplot as plt

In [0]:
x = np.arange(0,np.pi*2,0.05) # generate x values from 0 to 2pi in step of 0.05
y = np.sin(x) # compute y values
plt.plot(x,y, '-')
plt.ylabel('Intensity')
plt.xlabel('Time')
plt.axhline(ls='--',c='k')

Next, we simulate a first-order exponential decay of the form _Y = exp(-t/tau)_. We will create the function, `decay`, and use it to compute the _y_ values:

In [0]:
# define a function to y values for an array of x values
def decay(t, N, tau):
    return N*np.exp(-t/tau)

# create a vector of x values
t = np.linspace(0, 5, num=1000) # this generates the array t, with values from 0 to 5, in 1000 increments
# now compute the y values
N = 7 # intensity at time 0
tau = 0.5 # decay rate
data = decay(t, N, tau)
plt.plot(t,data)
plt.xlabel('Time')
plt.ylabel('Intensity')

## Simulate data, with noise...
Real experimental data is always noisy. We can simulated this noise using random numbers. We use the `decay` function and variables defined above...

In [0]:
# define a "noisy" function to y values for an array of x values
def noisy_decay(t, N, tau, sigma):
    return N*np.exp(-t/tau)+sigma*np.random.randn(*t.shape) 
# randn generates a "normal distribution" sample

tau = 1; N = 10; sigma = 0.5  # sigma is the standard deviation of the normal distribution
noisy_data = noisy_decay(t, N, tau, sigma)  
plt.plot(t,noisy_data, '.')
plt.title('Noisy Decay Data')
plt.xlabel('t')
plt.ylabel('y')

## Fitting noisy data
Now, let's fit these noisy data to a model (the epected decaying exponential), using `lmfit`

In [0]:
# install lmfit
!pip install lmfit  # this line not required if installed in recent session
from lmfit import Model, Parameter, report_fit  # import the desired components

In [0]:
model = Model(decay, independent_vars=['t'])
tau = 3; N = 5
#result = model.fit(noisy_data, t=t, N=10, tau=1)  # specified starting params
result = model.fit(noisy_data, t=t, N=N, tau=tau)  # specified starting params
result

In [0]:
result.plot('.')

In [0]:
result.params.pretty_print()

In the exercise above, we generated arrays _t_, _data_ and _noisy\_data_, but didn't place them in a pandas data frame. This can easily be accomplished by specifying a data frame with those arrays as the data:

In [0]:
import pandas as pd
d = {'t': t, 'data': noisy_data, 'init': result.init_fit, 'best_fit' : result.best_fit}  
# d contains several arrays, with headers
df = pd.DataFrame(data=d)
df.head()

In [0]:
# df.plot(x='t', y=['data','best_fit'], kind='scatter', c='orange', s=1)
df.plot(x='t', y=['data','best_fit','init'])